# Federated AutoEncoder  with Flex  for Anomaly Detection

First we do all the imports needed.

In [39]:
from flexanomalies.utils import AutoEncoder
from flexanomalies.utils.load_data import load_and_split_dot_mat, federate_data
from flexanomalies.pool.aggregators_favg import aggregate_ae
from flexanomalies.pool.primitives_deepmodel import (
    build_server_model_ae,
    copy_model_to_clients_ae,
    train_ae,
    set_aggregated_weights_ae,
    weights_collector_ae,
)
from flexanomalies.utils.save_results import save_experiments_results
from flex.pool import FlexPool
from flexanomalies.utils.metrics import *

In [40]:
model_params = {
    "epochs": 50,
    "input_dim": 9,
    "batch_size": 32,
    "neurons": [8, 4, 8],
    "hidden_act": ["relu", "relu", "relu"],
    "contamination": 0.1,
}

In [41]:
X_train, X_test, y_train, y_test = load_and_split_dot_mat(
    "flexanomalies/datasets/data/shuttle.mat", 0.3
)

In [42]:
model = AutoEncoder(**model_params)

In [43]:
flex_dataset = federate_data(5, X_train, y_train)

In [44]:
pool = FlexPool.client_server_pool(
    fed_dataset=flex_dataset,
    server_id="autoencoder_server",
    init_func=build_server_model_ae,
    model=model,
)

In [45]:
for i in range(3):
    print(f"\nRunning round: {i}\n")
    pool.servers.map(copy_model_to_clients_ae, pool.clients)
    pool.clients.map(train_ae)
    pool.aggregators.map(weights_collector_ae, pool.clients)
    pool.aggregators.map(aggregate_ae)
    pool.aggregators.map(set_aggregated_weights_ae, pool.servers)
output_model = pool.servers._models["autoencoder_server"]["model"]
output_model.evaluate(X_test, y_test)


Running round: 0

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
......dense_3
.........vars
............0
............1
...optimizer
......vars
.........0
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-12-14 11:46:34         2228
variables.h5                                   2023-12-14 11:46:34        17680
metadata.json                                  2023-12-14 11:46:34           64
Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2023-12-14 11:46:34         2228
variables.h5                                   2023-12-14 11:46:34        17680
metadata.json          

In [46]:
output_model.result_metrics_

{'Accuracy': '96.653%',
 'Precision': '0.682',
 'F1': '0.803',
 'Recall': '0.977',
 'AUC_ROC': '0.971'}

In [47]:
save_experiments_results(
    "autoencoder",
    output_model,
    "test_autoencoder_notebook",
    model_params,
    "shuttle.mat",
    5,
    3,
    0.3,
)